<a href="https://colab.research.google.com/github/chacha86/pythonai2/blob/main/%EB%84%A4%EC%9D%B4%EB%B2%84_%EB%9E%AD%ED%82%B9_%EB%89%B4%EC%8A%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# 1. 언론사 이름과 링크 가져오기
## 파이썬 웹 요청 라이브러리
import requests


## 파이썬이 브라우저인 척 하기 위함
headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
}

## 네이버 랭킹 뉴스 웹페이지 요청. 성공하면 응답이옴(r)
r = requests.get('https://news.naver.com/main/ranking/popularDay.naver', headers=headers)


## 응답 정보에는 많은 게 있지만 우린 text가져가면 된다. text는 단순 문자열이다.
ranking_news_html = r.text

## 뷰티풀숩으로 파싱
from bs4 import BeautifulSoup
soup = BeautifulSoup(ranking_news_html, 'html.parser')

# 하나만 선택 -> select_one
# 여러개 선택 -> select

box_list = soup.select("._officeCard .rankingnews_box")


journal_list = []

for box in box_list :
  atag = box.select_one('a')

  # 태그를 얻었을 때
  # 1. 텍스트
  # 2. 속성
  journal_link = atag['href']
  journal_name = atag.select_one('strong').text

  journal_dic = {
      'name' : journal_name,
      'link' : journal_link
  }

  journal_list.append(journal_dic)


def get_journal_by_name(name) :

  for journal in journal_list:
    if journal['name'] == name :
      return journal
  return None

get_journal_by_name("SBS")

{'name': 'SBS',
 'link': 'https://media.naver.com/press/055/ranking?type=popular'}

In [6]:
# 2. url을 넘기면 해당 url 페이지의 html을 파싱해서 soup를 만들어주는 함수
def get_page_by_url(url) :
  from bs4 import BeautifulSoup
  import requests

  headers = {
    'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/102.0.0.0 Safari/537.36'
  }
  r = requests.get(url, headers=headers)
  soup = BeautifulSoup(r.text, 'html.parser')

  return soup


In [7]:
# 3. page를 넘겨주면 해당 page에서 랭킹 뉴스 20개를 가져오는 함수
sbs = get_journal_by_name("SBS")
url = sbs['link']
news_page = get_page_by_url(url)

def get_rankingnews_by_page(page) :
  thumb_list = page.select('.as_thumb')

  news_list = []

  for thumb in thumb_list :
    link = thumb.select_one('a')['href'] # 본문링크
    rank = thumb.select_one('em').text # 랭킹
    title = thumb.select_one(".list_content strong").text  # 제목
    tmp = thumb.select_one(".list_content .list_view").text #  조회수,
    view = tmp[4:].strip()

    news_dic = {
        'link' : link,
        'rank' : rank,
        'title' : title,
        'view' : view
    }

    news_list.append(news_dic)

  return news_list

sbs_ranking_news = get_rankingnews_by_page(news_page)

In [8]:
import json
# 4. 파이썬 오브젝트를 json으로 저장해주는 함수
def save_to_json(filepath, obj) :
  with open(filepath, 'w') as f :
    json_str = json.dumps(obj, ensure_ascii=False)
    f.write(json_str)


In [9]:
# 5. json 파일을 파이썬 오브젝트로 불러오는 함수
def load_json(filepath) :
  with open(filepath, 'r') as f:
    json_str = f.read()
    return json.loads(json_str)

In [10]:
save_to_json('sbs_news.json', sbs_ranking_news)

In [11]:
sbs_news = load_json('sbs_news.json')
sbs_news

[{'link': 'https://n.news.naver.com/article/055/0001087319?ntype=RANKING',
  'rank': '1',
  'title': '고교생이 교실서 담임교사 5분간 폭행…학교, 퇴학 조치',
  'view': '18,585'},
 {'link': 'https://n.news.naver.com/article/055/0001087276?ntype=RANKING',
  'rank': '2',
  'title': "모텔방 잠그고 '찬송가 소란'…필로폰에 찌들어 있었다",
  'view': '12,141'},
 {'link': 'https://n.news.naver.com/article/055/0001087240?ntype=RANKING',
  'rank': '3',
  'title': '가짜 주민번호 대고 "맞다니까" 우기더니…딱 걸린 정체',
  'view': '8,032'},
 {'link': 'https://n.news.naver.com/article/055/0001087296?ntype=RANKING',
  'rank': '4',
  'title': '[포착] 달리던 택시에 불…"소화기!" 외치며 뛰어든 어린이집 교사',
  'view': '7,829'},
 {'link': 'https://n.news.naver.com/article/055/0001087285?ntype=RANKING',
  'rank': '5',
  'title': "단속 피하려 경찰 매달고 도망…오토바이 운전자 '징역 2년'",
  'view': '6,538'},
 {'link': 'https://n.news.naver.com/article/055/0001087324?ntype=RANKING',
  'rank': '6',
  'title': "검찰, '민주당 돈봉투' 재판서 현역 의원 이름 담긴 녹취록 공개",
  'view': '5,365'},
 {'link': 'https://n.news.naver.com/article/055/00010

In [47]:

# 실습 -> 3개의 언론사 (SBS, MBC, KBS)의 랭킹 뉴스를 최근 5일치 가져와서 각각 저장.
# 언론사와 n을 넘기면 해당 언론사의 랭킹뉴스를 n일치 가져오기
# 언론사별로 폴더를 나눠서 저장

def get_n_latest_news(journal, n) :

  # 1. 오늘 날짜로 부터 n일 전까지의 날짜 얻기


  # 2. 얻은 날짜로 날짜별 언론사 링크 만들기

  # 3. 랭킹뉴스 가져오는 함수를 이용해 n일치 뉴스 가져오기

  # 4. 언론사별로 폴더 나눠서 저장하기


In [2]:
from datetime import datetime, timedelta


def get_n_dates(n) :
  date = datetime.today()
  date_list = []

  for i in range(n) :

    result = date - timedelta(days=i)
    date_list.append(result.strftime("%Y%m%d"))

  return date_list

In [20]:
def get_n_latest_news(journal, n) :
  sbs = get_journal_by_name(journal)
  date_list = get_n_dates(n)

  total_news_list = {}

  for date in date_list :
    date_link = sbs['link'] + '&date=' + date
    news_page = get_page_by_url(url)
    news_list = get_rankingnews_by_page(news_page)
    total_news_list[date] = news_list

  return total_news_list

# get_n_latest_news('SBS', 5)
kbs_10days = get_n_latest_news('KBS', 10)



In [23]:
kbs_10days['20230904']

[{'link': 'https://n.news.naver.com/article/055/0001087319?ntype=RANKING',
  'rank': '1',
  'title': '고교생이 교실서 담임교사 5분간 폭행…학교, 퇴학 조치',
  'view': '18,585'},
 {'link': 'https://n.news.naver.com/article/055/0001087276?ntype=RANKING',
  'rank': '2',
  'title': "모텔방 잠그고 '찬송가 소란'…필로폰에 찌들어 있었다",
  'view': '12,141'},
 {'link': 'https://n.news.naver.com/article/055/0001087240?ntype=RANKING',
  'rank': '3',
  'title': '가짜 주민번호 대고 "맞다니까" 우기더니…딱 걸린 정체',
  'view': '8,032'},
 {'link': 'https://n.news.naver.com/article/055/0001087296?ntype=RANKING',
  'rank': '4',
  'title': '[포착] 달리던 택시에 불…"소화기!" 외치며 뛰어든 어린이집 교사',
  'view': '7,829'},
 {'link': 'https://n.news.naver.com/article/055/0001087285?ntype=RANKING',
  'rank': '5',
  'title': "단속 피하려 경찰 매달고 도망…오토바이 운전자 '징역 2년'",
  'view': '6,538'},
 {'link': 'https://n.news.naver.com/article/055/0001087324?ntype=RANKING',
  'rank': '6',
  'title': "검찰, '민주당 돈봉투' 재판서 현역 의원 이름 담긴 녹취록 공개",
  'view': '5,365'},
 {'link': 'https://n.news.naver.com/article/055/00010